


# Import the  Library

In [3]:
import sys
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix

import warnings
warnings.filterwarnings("ignore") 

# Load the data and data proprocessing 

In [4]:
data = pd.read_csv("data.csv",engine='python',encoding='gbk')
data.head()

,stu_id,time,course_index,name,type,type_id
0,0,2017/6/1 9:02,0,中国建筑史（上）,艺术·设计 历史,20.0
1,0,2017/7/4 7:52,1,外国工艺美术史,艺术·设计,20.0
2,0,2017/7/4 7:55,2,心理学概论,社科·法律,13.0
3,0,2017/7/20 5:35,3,经济学原理,经管·会计,10.0
4,0,2017/11/14 5:36,4,公司金融,经管·会计,10.0


### select the stu_id and course_id and add a column rating 1 to show it has chosen this course

In [5]:
df = data[['stu_id','course_index']]
df['rating'] = 1
df.head()

,stu_id,course_index,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


# Creates the sparse student-course interaction matrix 

In [6]:
def create_matrix(data, stu_col, courses_col, ratings_col):
    """
    creates the sparse student-course interaction matrix 
    """
    
    for col in (courses_col, stu_col, ratings_col):
        data[col] = data[col].astype('category')

    ratings = csr_matrix((data[ratings_col],
                          (data[stu_col].cat.codes, data[courses_col].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data

In [7]:
courses_col = 'course_index'
stu_col = 'stu_id'
ratings_col = 'rating'
X, df = create_matrix(df, stu_col, courses_col, ratings_col)
X

<82535x1302 sparse matrix of type '<class 'numpy.int64'>'
	with 458453 stored elements in Compressed Sparse Row format>

# Split date into training set and testing set

split the  student-course interactions matrix into train and test set by removing some of the interactions from every student and pretend |that we never seen them

In [8]:
def create_train_test(ratings):
    """
    split the  student-course interactions matrix into train and test set
    by removing some of the interactions from every student and pretend
    that we never seen them
    """
   
    # Dictionary Of Keys based sparse matrix is more efficient
    # for constructing sparse matrices incrementally compared with csr_matrix
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
    # select the most recent course as the test set, the remaining as the train set
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        test_index = split_index[-1:]
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
    
    train, test = train.tocsr(), test.tocsr()
    return train, test

In [9]:
X_train, X_test = create_train_test(X)
X_train

<82535x1302 sparse matrix of type '<class 'numpy.int64'>'
	with 375918 stored elements in Compressed Sparse Row format>

# Construct the BPR class 

Bayesian Personalized Ranking (BPR) is derived from personalized ranking, which provides users with item recommendations of a ranked list of items. The ranked list of items is calculated from the users’ implicit behavior. BPR is based on matrix factorization. The chosen courses can be seen as positive datasets while the remaining courses can be a mixture of negative and missing values. Typically, the course recommenders output the personalized score $X_{ui}$ (u is a student and i is a course) based on the preference of the student for the courses, and courses are sorted from the predicted score. The machine learning model of course recommenders provides the training data by giving pairs (u, i) $\in$ S as a positive class label and all other combinations in (U × I) $\backslash$ S as the negative one. Here all the negative user-course pairs are replaced by 0.

This algorithm is optimized by SGD.

In [11]:
class BPR:
    """
    Bayesian Personalized Ranking (BPR) for implicit feedback data

    Parameters
    ----------
    learning_rate : float, default 0.01
        learning rate for gradient descent

    n_factors : int, default 20
        Number/dimension of latent factors

    n_iters : int, default 15
        Number of iterations to train the algorithm
        
    batch_size : int, default 1000
        batch size for batch gradient descent

    reg : int, default 0.01
        Regularization term for the latent factors

    seed : int, default 1234
        Seed for the randomly initialized student, courses latent factors

    verbose : bool, default True
        Whether to print progress bar while training

    Attributes
    ----------
    student_factors : 2d ndarray, shape [n_students, n_factors]
        student latent factors learnt

    course_factors : 2d ndarray, shape [n_students, n_factors]
        course latent factors learnt

    References
    ----------
    S. Rendle, C. Freudenthaler, Z. Gantner, L. Schmidt-Thieme 
    Bayesian Personalized Ranking from Implicit Feedback
    - https://arxiv.org/abs/1205.2618
    """
    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 batch_size = 1000, reg = 0.01, seed = 1234, verbose = True):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings):
        """
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_students, n_courses]
            sparse matrix of student-course interactions
        """
        indptr = ratings.indptr
        indices = ratings.indices
        n_students, n_courses = ratings.shape
        
        # ensure batch size makes sense, since the algorithm involves for each step randomly sample a student, 
        # thus the batch size should be smaller than the total number of students or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_students < batch_size:
            batch_size = n_students
            sys.stderr.write('WARNING: Batch size is greater than number of students,'
                             'switching to a batch size of {}\n'.format(n_students))

        batch_iters = n_students // batch_size
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.student_factors = rstate.normal(size = (n_students, self.n_factors))
        self.course_factors = rstate.normal(size = (n_courses, self.n_factors))
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_students, n_courses, indices, indptr)
                sampled_students, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_students, sampled_pos_items, sampled_neg_items)

        return self
    
    def _sample(self, n_students, n_courses, indices, indptr):
        """sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
        sampled_students = np.random.choice(n_students, size = self.batch_size, replace = False)

        for idx, user in enumerate(sampled_students):
            pos_items = indices[indptr[user]:indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_courses)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_courses)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_students, sampled_pos_items, sampled_neg_items
                
    def _update(self, u, i, j):
        """
        update according to the bootstrapped user u, 
        positive item i and negative item j
        """
        stu_u = self.student_factors[u]
        course_i = self.course_factors[i]
        course_j = self.course_factors[j]
        
        # decompose the estimator, compute the difference between
        # the score of the positive items and negative items; a
        # naive implementation might look like the following:
        # r_ui = np.diag(stu_u.dot(course_i.T))
        # r_uj = np.diag(stu_u.dot(course_j.T))
        # r_uij = r_ui - r_uj
        
        # however, we can do better, so
        # for batch dot product, instead of doing the dot product
        # then only extract the diagonal element (which is the value
        # of that current batch), we perform a hadamard product, 
        # i.e. matrix element-wise product then do a sum along the column will
        # be more efficient since it's less operations
        # r_ui = np.sum(stu_u * course_i, axis = 1)
        #
        # then we can achieve another speedup by doing the difference
        # on the positive and negative item up front instead of computing
        # r_ui and r_uj separately, these two idea will speed up the operations
        # from 1:14 down to 0.36
        r_uij = np.sum(stu_u * (course_i - course_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_u = sigmoid_tiled * (course_j - course_i) + self.reg * stu_u
        grad_i = sigmoid_tiled * -stu_u + self.reg * course_i
        grad_j = sigmoid_tiled * stu_u + self.reg * course_j
        self.student_factors[u] -= self.learning_rate * grad_u
        self.course_factors[i] -= self.learning_rate * grad_i
        self.course_factors[j] -= self.learning_rate * grad_j
        return self

    def predict(self):
        """
        Obtain the predicted ratings for every students and courses
        by doing a dot product of the learnt student and courses vectors.
        The result will be cached to avoid re-computing it every time
        we call predict, thus there will only be an overhead the first
        time we call it. Note, ideally you probably don't need to compute
        this as it returns a dense matrix and may take up huge amounts of
        memory for large datasets
        """
        if self._prediction is None:
            self._prediction = self.student_factors.dot(self.course_factors.T)

        return self._prediction

    def _predict_stu(self, student):
        """
        returns the predicted ratings for the specified student,
        this is mainly used in computing evaluation metric
        """
        stu_pred = self.student_factors[student].dot(self.course_factors.T)
        return stu_pred

    def recommend(self, ratings, N = 5):
        """
        Returns the top N ranked courses for given student id, excluding the ones that the student already chosen
        """
        n_students = ratings.shape[0]
        recommendation = np.zeros((n_students, N), dtype = np.uint32)
        for student in range(n_students):
            top_n = self._recommend_stu(ratings, student, N)
            recommendation[student] = top_n

        return recommendation

    def _recommend_stu(self, ratings, student, N):
        """the top-N ranked courses for a given student"""
        scores = self._predict_stu(student)

        # compute the top N courses, removing the courses that the student already chosen
        # from the result and ensure that we don't get out of bounds error when 
        # we ask for more recommendations than that are available
        chosen = set(ratings[student].indices)
        count = N + len(chosen)
        if count < scores.shape[0]:
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
        else:
            best = np.argsort(scores)[::-1]

        top_n = list(islice((rec for rec in best if rec not in chosen), N))
        return top_n


# Define the function to calculate the hit rate and NDCG

In [12]:
def hit_rate_cal(model,X_train, X_test,N):
    hit = 0
    n_students = X_train.shape[0]
    recommend_courses = model.recommend(X_train, N)
    for student in range(n_students):
        test_course = X_test[student].indices
        if test_course in recommend_courses[student]:
            hit += 1
    hit_rate = hit /n_students
    return hit_rate

In [13]:
def NDCG_cal(model,X_train, X_test,N):
    NDCG = 0
    n_students = X_train.shape[0]
    recommend_courses = model.recommend(X_train, N)
    for student in range(n_students):
        test_course = X_test[student].indices
        if test_course in recommend_courses[student]:
            i = np.where(recommend_courses[student] == test_course)[0][0]
            if i == 0:
                NDCG += 1
            else:
                NDCG += 1/np.log2(i+1)
    NDCG = NDCG /n_students
    return NDCG

# Evaluate the model

### Find the relationship between hit rate and the number of latent dimension

In [ ]:
hit_rate = []
for i in range(10,81,10):
    bpr_params = {'reg': 0.01,
              'learning_rate': 0.2,
              'n_iters': 160,
              'n_factors': i,
              'batch_size': 100}
    bpr = BPR(**bpr_params)
    bpr.fit(X_train)
    hit_rate.append(hit_rate_cal(bpr,X_train, X_test,10))
hit_rate

BPR: 100%|██████████| 160/160 [06:46<00:00,  2.54s/it]


[0.1491973102320228,
 0.15335312291755013,
 0.1505300781486642,
 0.14271521172835766,
 0.13131398800508876,
 0.12508632701278247,
 0.12393530017568305,
 0.13251347913006603]

### Find the relationship between NDCG and the number of latent dimension

In [ ]:
ndcg = []
for i in range(10,81,10):
    bpr_params = {'reg': 0.01,
              'learning_rate': 0.2,
              'n_iters': 160,
              'n_factors': i,
              'batch_size': 100}
    bpr = BPR(**bpr_params)
    bpr.fit(X_train)
    ndcg.append(NDCG_cal(bpr,X_train, X_test,10))
ndcg

BPR: 100%|██████████| 160/160 [06:44<00:00,  2.53s/it]


[0.08430180938066924,
 0.08687534561515821,
 0.09629560625117531,
 0.07371647078542436,
 0.08960175124571808,
 0.07815336074488172,
 0.07853411779639283,
 0.0746160539647747]

### Find the relationship between NDCG and the number of recommended courses

In [14]:
cutoff = []
bpr_params = {'reg': 0.01,
            'learning_rate': 0.2,
            'n_iters': 160,
            'n_factors': 30,
            'batch_size': 100}
bpr = BPR(**bpr_params)
bpr.fit(X_train)
for N in range(5,16):
    cutoff.append(NDCG_cal(bpr,X_train, X_test,N))
cutoff

BPR: 100%|██████████| 160/160 [07:12<00:00,  2.70s/it]


[0.07101090603124147,
 0.075730852440463,
 0.07950720536612653,
 0.08267757753146958,
 0.08537986950321003,
 0.08781991393404849,
 0.0900158749942224,
 0.0921923971605701,
 0.09405215691964033,
 0.0960219886539096,
 0.09763771690163396]

### Find the relationship between hit rate and the number of recommended courses

In [13]:
cutoff_HIT = []
bpr_params = {'reg': 0.01,
            'learning_rate': 0.2,
            'n_iters': 160,
            'n_factors': 30,
            'batch_size': 100}
bpr = BPR(**bpr_params)
bpr.fit(X_train)
for N in range(5,16):
    cutoff_HIT.append(hit_rate_cal(bpr,X_train, X_test,N))
cutoff_HIT

BPR: 100%|██████████| 160/160 [07:15<00:00,  2.72s/it]


[0.0898406736536015,
 0.10113285272914521,
 0.1110922638880475,
 0.12043375537650694,
 0.12874538074756164,
 0.13739625613376144,
 0.14599866723208335,
 0.15289271218271036,
 0.16035621251590235,
 0.1676864360574302,
 0.17514993639062215]